In [1]:
import pandas as pd
import os
import numpy as np
import scipy.stats as stats

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [2]:
PATH = 'ml/results_rf_combined.csv'

In [3]:
results = pd.read_csv(PATH, index_col=0)

chunk_sizes = results.chunk_size.unique()

complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)
grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['auc']].median()

auc
restriction modality      chunk_size          
complete    morphological 0.0         0.995410
                          25.0        0.999139
                          50.0        0.999426
                          100.0       0.999426
                          200.0       0.999426
                          400.0       0.999426
                          800.0       0.999713
                          1600.0      0.998566
            spatial       0.0         0.827883
                          25.0        0.962995
                          50.0        0.945640
                          100.0       0.943488
                          200.0       0.923982
                          400.0       0.931010
                          800.0       0.903041
                          1600.0      0.906770
            temporal      0.0         0.975043
                          25.0        0.975043
                          50.0        0.973465
                          100.0       0.967011
                          200.0       0.969880
                          400.0       0.978199
                          800.0       0.975760
                          1600.0      0.977338

## WF

In [4]:
wf_res = complete[complete.modality == 'morphological']
grouped_wf = wf_res.groupby(by=['chunk_size'])
grouped_wf[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0.0        0.25  0.978199
           0.50  0.995410
           0.75  1.000000
25.0       0.25  0.974039
           0.50  0.999139
           0.75  1.000000
50.0       0.25  0.986517
           0.50  0.999426
           0.75  1.000000
100.0      0.25  0.971888
           0.50  0.999426
           0.75  1.000000
200.0      0.25  0.973537
           0.50  0.999426
           0.75  1.000000
400.0      0.25  0.972174
           0.50  0.999426
           0.75  1.000000
800.0      0.25  0.972533
           0.50  0.999713
           0.75  1.000000
1600.0     0.25  0.969449
           0.50  0.998566
           0.75  1.000000

In [5]:
wf_aucs = [wf_res.auc[wf_res.chunk_size == cs] for cs in chunk_sizes]
wf_aucs_0 = wf_aucs[0]
print(f"comparing CS=0 to baseline for WF models:", stats.wilcoxon(wf_aucs_0-0.5, alternative='greater'))
for cs_aucs, cs in zip(wf_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for WF models:", stats.wilcoxon(cs_aucs, wf_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / wf_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(es)}")

comparing CS=0 to baseline for WF models: WilcoxonResult(statistic=1275.0, pvalue=3.130562367449191e-10)
comparing CS=0 to CS=25.0 for WF models: WilcoxonResult(statistic=401.5, pvalue=0.01530049348262902)
The effect size is: 1.0
comparing CS=0 to CS=50.0 for WF models: WilcoxonResult(statistic=424.0, pvalue=0.0013850673244110833)
The effect size is: 1.0011490953719466
comparing CS=0 to CS=100.0 for WF models: WilcoxonResult(statistic=364.0, pvalue=0.06782310535060246)
The effect size is: 1.0
comparing CS=0 to CS=200.0 for WF models: WilcoxonResult(statistic=412.0, pvalue=0.009382108273501803)
The effect size is: 1.0002870264064292
comparing CS=0 to CS=400.0 for WF models: WilcoxonResult(statistic=320.5, pvalue=0.14535109418446351)
The effect size is: 1.0
comparing CS=0 to CS=800.0 for WF models: WilcoxonResult(statistic=362.0, pvalue=0.07264069940082778)
The effect size is: 1.0
comparing CS=0 to CS=1600.0 for WF models: WilcoxonResult(statistic=267.5, pvalue=0.5918458180892879)
The ef

largest effect size was found for 50-spikes chunks, improvement:

In [6]:
# temp here stands for tempporary not temporal
temp = complete[complete.modality == 'morphological']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_50 = temp[temp.chunk_size == 50].auc.to_numpy()
diff = 100 * (temp_50 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 50: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

Q25, Q50 and Q75  for chunk size = 50: [0.         0.11490954 0.55336064]


## spatial

In [7]:
spat_res = complete[complete.modality == 'spatial']
grouped_spat = spat_res.groupby(by=['chunk_size'])
grouped_spat[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0.0        0.25  0.802065
           0.50  0.827883
           0.75  0.848609
25.0       0.25  0.949369
           0.50  0.962995
           0.75  0.974900
50.0       0.25  0.931727
           0.50  0.945640
           0.75  0.960843
100.0      0.25  0.927711
           0.50  0.943488
           0.75  0.955250
200.0      0.25  0.898881
           0.50  0.923982
           0.75  0.946357
400.0      0.25  0.899742
           0.50  0.931010
           0.75  0.947935
800.0      0.25  0.865175
           0.50  0.903041
           0.75  0.928141
1600.0     0.25  0.870697
           0.50  0.906770
           0.75  0.927352

In [8]:
spat_aucs = [spat_res.auc[spat_res.chunk_size == cs] for cs in chunk_sizes]
spat_aucs_0 = spat_aucs[0]
print(f"comparing CS=0 to baseline for Spatial models:", stats.wilcoxon(spat_aucs_0-0.5, alternative='greater'))
for cs_aucs, cs in zip(spat_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, spat_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / spat_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(es)}")

comparing CS=0 to baseline for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7759023791403374e-10)
comparing CS=0 to CS=25.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7776104381903367e-10)
The effect size is: 1.163251630387498
comparing CS=0 to CS=50.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.1488755210325325
comparing CS=0 to CS=100.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.1385109151311457
comparing CS=0 to CS=200.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.7776104381903367e-10)
The effect size is: 1.106771359149762
comparing CS=0 to CS=400.0 for Spatial models: WilcoxonResult(statistic=1275.0, pvalue=3.778464727931783e-10)
The effect size is: 1.1185841571399644
comparing CS=0 to CS=800.0 for Spatial models: WilcoxonResult(statistic=1252.5, pvalue=1.4528794200696912e-09)
The effect size is: 1.0927681673621

largest effect size was found for 25-spikes chunks, improvement:

In [9]:
temp = complete[complete.modality == 'spatial']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_25 = temp[temp.chunk_size == 25].auc.to_numpy()
diff = 100 * (temp_25 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 25: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

Q25, Q50 and Q75  for chunk size = 25: [13.27175918 16.32516304 19.63062307]


## Spike-timing

In [10]:
temp_res = complete[complete.modality == 'temporal']
grouped_temp = temp_res.groupby(by=['chunk_size'])
grouped_temp[['auc']].quantile(q=[0.25, 0.5, 0.75])

auc
chunk_size               
0.0        0.25  0.961059
           0.50  0.975043
           0.75  0.986661
25.0       0.25  0.959983
           0.50  0.975043
           0.75  0.981641
50.0       0.25  0.960413
           0.50  0.973465
           0.75  0.980493
100.0      0.25  0.949297
           0.50  0.967011
           0.75  0.983864
200.0      0.25  0.955321
           0.50  0.969880
           0.75  0.982788
400.0      0.25  0.961704
           0.50  0.978199
           0.75  0.985513
800.0      0.25  0.962278
           0.50  0.975760
           0.75  0.985298
1600.0     0.25  0.965003
           0.50  0.977338
           0.75  0.987378

In [11]:
temp_aucs = [temp_res.auc[temp_res.chunk_size == cs] for cs in chunk_sizes]
temp_aucs_0 = temp_aucs[0]
print(f"comparing CS=0 to baseline for Temporal models:", stats.wilcoxon(temp_aucs_0 - 0.5, alternative='greater'))
for cs_aucs, cs in zip(temp_aucs[1:], chunk_sizes[1:]):
    print(f"comparing CS=0 to CS={cs} for Spatial models:", stats.wilcoxon(cs_aucs, temp_aucs_0, alternative='greater'))
    es = cs_aucs.to_numpy() / temp_aucs_0.to_numpy()
    print(f"The effect size is: {np.median(es)}")

comparing CS=0 to baseline for Temporal models: WilcoxonResult(statistic=1275.0, pvalue=3.7750486097697307e-10)
comparing CS=0 to CS=25.0 for Spatial models: WilcoxonResult(statistic=347.0, pvalue=0.997479675102897)
The effect size is: 0.9948261491643633
comparing CS=0 to CS=50.0 for Spatial models: WilcoxonResult(statistic=355.0, pvalue=0.9947890515441402)
The effect size is: 0.9959098022721156
comparing CS=0 to CS=100.0 for Spatial models: WilcoxonResult(statistic=210.0, pvalue=0.9999816096728339)
The effect size is: 0.9925021612654501
comparing CS=0 to CS=200.0 for Spatial models: WilcoxonResult(statistic=291.5, pvalue=0.998821835645379)
The effect size is: 0.994857009561388
comparing CS=0 to CS=400.0 for Spatial models: WilcoxonResult(statistic=672.0, pvalue=0.3695483637804753)
The effect size is: 1.0008792684264534
comparing CS=0 to CS=800.0 for Spatial models: WilcoxonResult(statistic=576.5, pvalue=0.6398692217084291)
The effect size is: 1.0002912111475017
comparing CS=0 to CS=16

largest effect size was found for 1600-spikes chunks, improvement:

In [12]:
temp = complete[complete.modality == 'temporal']
temp_0 = temp[temp.chunk_size == 0].auc.to_numpy()
temp_1600 = temp[temp.chunk_size == 1600].auc.to_numpy()
diff = 100 * (temp_1600 - temp_0) / temp_0
print(f"Q25, Q50 and Q75  for chunk size = 1600: {np.quantile(diff, q=[0.25, 0.5, 0.75])}")

Q25, Q50 and Q75  for chunk size = 1600: [-0.3928307   0.27547579  0.74311007]


## Cross-comparisons

In [13]:
temp_1600 = complete[complete.chunk_size == 1600]
temp_1600 = temp_1600[temp_1600.modality == 'temporal'].auc

wf_50 = complete[complete.chunk_size == 50]
wf_50 = wf_50[wf_50.modality == 'morphological'].auc

spat_25 = complete[complete.chunk_size == 25]
spat_25 = spat_25[spat_25.modality == 'spatial'].auc

print('comparing WF and temporal:', stats.wilcoxon(wf_50, temp_1600))
print('comparing WF and spatial:', stats.wilcoxon(wf_50, spat_25))
print('comparing spatial and temporal:', stats.wilcoxon(spat_25, temp_1600))

comparing WF and temporal: WilcoxonResult(statistic=51.0, pvalue=1.496094997239071e-08)
comparing WF and spatial: WilcoxonResult(statistic=10.0, pvalue=1.3797825872143393e-09)
comparing spatial and temporal: WilcoxonResult(statistic=159.5, pvalue=6.600503960896184e-06)
